HSE, Applied Time Series Forecasitng , Winter 2023-24

<font color="green"> Lesson #13: Bonus Topics </font>

<span style="color:black; font-size: 12pt"></span>

Alexey Romanenko,
<font color="blue">alexromsput@gmail.com</font>

**Key words:**
 demand restoration, value from forecasting

**Your feedback:**  please provide you feedback  <a href="https://forms.gle/bEV5iwmTqdM9rUoy8"> here </a>

In [3]:
import numpy as np
import pandas as pd
from copy import deepcopy

# don't forget to upload utils file to the colab session
import warnings, pylab

warnings.filterwarnings('ignore')

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

# Using TS forecasting packages. Online Demo

## Online Demo based on HW3

In [8]:
# загружаем данные
ts_hw3 = pd.read_csv('train.csv', sep=',')
ts_hw3['period_start_dt'] = pd.to_datetime(ts_hw3['period_start_dt'], format= "%Y-%m-%d") #dayfirst=True)
ts_hw3.set_index(['period_start_dt'], inplace=True)
ts_hw3 = ts_hw3.sort_index() # must have
ts_hw3.head()

,Unnamed: 0,product_rk,store_location_rk,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
period_start_dt,,,,,,,,,,
2016-12-19,0,40369,309,29.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21361,40369,1034,20.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21362,40370,1034,31.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21363,40372,1034,11.0,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-19,21364,40373,1034,14.0,NaN,NaN,NaN,NaN,NaN,NaN


## Using of python packages for time series forecasting

https://github.com/aromanenko/ATSF/wiki/List-of-packages-for-Time-Series-Forecasting-in-Python

# Business Value from Forecasting
 Business case: your company launched new Demand Forecasting System. What couldn't be as a business-value due to this implementation?
  
  a) free money due less overforecasting or less underforecasting
  
  b) free resources of analytics (less cost of support resources)
  
  c) increase transparancy and consistency in decision making processes (new money will come)
  
  d) speed up decision making process (more money due to fast reaction, more frequent decision making process)